In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# %matplotlib inline

In [2]:
# Load data
df = pd.read_csv("traffic_weather_full2020_cleaned.csv")
df.head()

,5 Minutes,Flow,(mph),week_day,holidays,Hour,Temperature,Dew Point,Humidity,Wind Speed,...,Condition_Fog,Condition_Haze,Condition_Heavy Rain,Condition_Light Rain,Condition_Mostly Cloudy,Condition_Mostly Cloudy / Windy,Condition_Partly Cloudy,Condition_Partly Cloudy / Windy,Condition_Rain,Condition_Thunder in the Vicinity
0,2020-02-01 00:15:00,337,65.4,6,1,0,55.0,29.0,37.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-02-01 00:20:00,302,66.3,6,1,0,55.0,29.0,37.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-02-01 00:25:00,282,65.3,6,1,0,55.0,29.0,37.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-02-01 00:30:00,302,66.2,6,1,0,55.0,29.0,37.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-02-01 00:35:00,277,66.2,6,1,0,55.0,29.0,37.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Model Implementation

In [3]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score, 
    explained_variance_score, max_error
)

#### Feature Selection

In [4]:
columns = df.columns
columns

Index(['5 Minutes', 'Flow', '(mph)', 'week_day', 'holidays', 'Hour',
       'Temperature', 'Dew Point', 'Humidity', 'Wind Speed', 'Wind Gust',
       'Pressure', 'Precip.', 'Day_of_Month', 'Flow_Lag1', 'Flow_Lag2',
       'Flow_Lag3', 'Wind_CALM', 'Wind_E', 'Wind_ENE', 'Wind_ESE', 'Wind_N',
       'Wind_NE', 'Wind_NNE', 'Wind_NNW', 'Wind_NW', 'Wind_S', 'Wind_SE',
       'Wind_SSE', 'Wind_SSW', 'Wind_SW', 'Wind_VAR', 'Wind_W', 'Wind_WNW',
       'Wind_WSW', 'Condition_Blowing Dust', 'Condition_Cloudy',
       'Condition_Cloudy / Windy', 'Condition_Fair', 'Condition_Fair / Windy',
       'Condition_Fog', 'Condition_Haze', 'Condition_Heavy Rain',
       'Condition_Light Rain', 'Condition_Mostly Cloudy',
       'Condition_Mostly Cloudy / Windy', 'Condition_Partly Cloudy',
       'Condition_Partly Cloudy / Windy', 'Condition_Rain',
       'Condition_Thunder in the Vicinity'],
      dtype='object')

In [5]:
features = ['(mph)', 'week_day', 'holidays', 'Hour',
       'Temperature', 'Humidity', 'Wind Speed', 'Wind Gust',
       'Precip.', 'Flow_Lag1', 'Flow_Lag2','Flow_Lag3']

In [6]:
# Select all fatures
X_full = df.drop(['5 Minutes', 'Flow'], axis=1)

# Remove low correlated features
X_partial = df[features]
y = df['Flow']

#### Data spliting

In [7]:
split_point = int(0.8 * len(df))  # Calculate the 80% split index
X_train_full, X_test_full = X_full[:split_point], X_full[split_point:]
X_train_partial, X_test_partial = X_partial[:split_point], X_partial[split_point:]
y_train, y_test = y[:split_point], y[split_point:]


In [8]:
scaler = StandardScaler()

# Fit the scaler on the training set and transform both training and test sets
X_train_full_scaled = scaler.fit_transform(X_train_full)
X_test_full_scaled = scaler.transform(X_test_full)

X_train_partial_scaled = scaler.fit_transform(X_train_partial)
X_test_partial_scaled = scaler.transform(X_test_partial)

#### SVM using RBF kernel

In [10]:
# Step 2: Create the SVM model with an RBF kernel
svm_model_full = SVR(kernel='rbf')

# Step 3: Train the model
svm_model_full.fit(X_train_full_scaled, y_train)

# Step 4: Make predictions on the test set
y_pred_full = svm_model_full.predict(X_test_full_scaled)

# Step 4: Evaluate the model with various metrics
mae_svm_full = mean_absolute_error(y_test, y_pred_full)
mse_svm_full = mean_squared_error(y_test, y_pred_full)
rmse_svm_full = mean_squared_error(y_test, y_pred_full, squared=False)
r2_svm_full = r2_score(y_test, y_pred_full)
mape_svm_full = (abs((y_test - y_pred_full) / y_test).mean()) * 100
evs_svm_full = explained_variance_score(y_test, y_pred_full)
max_err_svm_full = max_error(y_test, y_pred_full)

# Print the evaluation metrics
print(f"Time-based splitting (full features):")
print(f"SVM Mean Absolute Error (MAE): {mae_svm_full:.4f}")
print(f"SVM Mean Squared Error (MSE): {mse_svm_full:.4f}")
print(f"SVM Root Mean Squared Error (RMSE): {rmse_svm_full:.4f}")
print(f"SVM R-squared (R2): {r2_svm_full:.4f}")
print(f"SVM Mean Absolute Percentage Error (MAPE): {mape_svm_full:.4f}%")
print(f"SVM Explained Variance Score: {evs_svm_full:.4f}")
print(f"SVM Max Error: {max_err_svm_full:.4f}")

Time-based splitting (full features):
SVM Mean Absolute Error (MAE): 28.8730
SVM Mean Squared Error (MSE): 1795.2339
SVM Root Mean Squared Error (RMSE): 42.3702
SVM R-squared (R2): 0.9379
SVM Mean Absolute Percentage Error (MAPE): 8.9377%
SVM Explained Variance Score: 0.9383
SVM Max Error: 286.9697


In [11]:
# Step 2: Create the SVM model with an RBF kernel
svm_model_partial = SVR(kernel='rbf')

# Step 3: Train the model
svm_model_partial.fit(X_train_partial_scaled, y_train)

# Step 4: Make predictions on the test set
y_pred_partial = svm_model_partial.predict(X_test_partial_scaled)

# Step 4: Evaluate the model with various metrics
mae_svm_partial = mean_absolute_error(y_test, y_pred_partial)
mse_svm_partial = mean_squared_error(y_test, y_pred_partial)
rmse_svm_partial = mean_squared_error(y_test, y_pred_partial, squared=False)
r2_svm_partial = r2_score(y_test, y_pred_partial)
mape_svm_partial = (abs((y_test - y_pred_partial) / y_test).mean()) * 100
evs_svm_partial = explained_variance_score(y_test, y_pred_partial)
max_err_svm_partial = max_error(y_test, y_pred_partial)

# Print the evaluation metrics
print(f"Time-based splitting (partial features):")
print(f"SVM Mean Absolute Error (MAE): {mae_svm_partial:.4f}")
print(f"SVM Mean Squared Error (MSE): {mse_svm_partial:.4f}")
print(f"SVM Root Mean Squared Error (RMSE): {rmse_svm_partial:.4f}")
print(f"SVM R-squared (R2): {r2_svm_partial:.4f}")
print(f"SVM Mean Absolute Percentage Error (MAPE): {mape_svm_partial:.4f}%")
print(f"SVM Explained Variance Score: {evs_svm_partial:.4f}")
print(f"SVM Max Error: {max_err_svm_partial:.4f}")

Time-based splitting (partial features):
SVM Mean Absolute Error (MAE): 22.9779
SVM Mean Squared Error (MSE): 962.6142
SVM Root Mean Squared Error (RMSE): 31.0260
SVM R-squared (R2): 0.9667
SVM Mean Absolute Percentage Error (MAPE): 6.3963%
SVM Explained Variance Score: 0.9670
SVM Max Error: 288.9711


Above results shows that the model using partial features performs better, so I will use partial features for subsequent hyperparameter tuning.

In [12]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.1, 1, 10],        # Regularization parameter
    'gamma': [0.001, 0.01, 0.5],  # Kernel coefficient
    'epsilon': [0.01, 0.1, 1],  # Epsilon in the epsilon-SVR model
    'kernel': ['rbf']              # Use RBF kernel
}

# Set up GridSearchCV with SVR
grid_search = GridSearchCV(
    SVR(),
    param_grid,
    scoring='neg_mean_squared_error',  # Using negative MSE for scoring
    cv=3,  # 3-fold cross-validation
    verbose=1
)

# Run grid search on the scaled training data
grid_search.fit(X_train_partial_scaled, y_train)



Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=3, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10], 'epsilon': [0.01, 0.1, 1],
                         'gamma': [0.001, 0.01, 0.5], 'kernel': ['rbf']},
             scoring='neg_mean_squared_error', verbose=1)

In [13]:

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score (negative MSE): ", grid_search.best_score_)

# Train the model using the best parameters
best_svm_model = grid_search.best_estimator_

best_svm_model.fit(X_train_partial_scaled, y_train) 

Best parameters found:  {'C': 10, 'epsilon': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
Best cross-validation score (negative MSE):  -984.300484420333


SVR(C=10, gamma=0.001)

In [14]:
import joblib
joblib_file = "best_svm_model.pkl"  
joblib.dump(best_svm_model, joblib_file)



['best_svm_model.pkl']

In [10]:
# Load from file

import joblib
joblib_file = "best_svm_model.pkl"  
joblib_SVM_model = joblib.load(joblib_file)


joblib_SVM_model

c:\Users\dmhan\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator SVR from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


SVR(C=10, gamma=0.001)

In [ ]:
# Use the Reloaded Joblib Model to 
# Calculate the accuracy score and predict target values

# Calculate the Score 
score = joblib_SVM_model.score(X_test_partial_scaled, y_test)  
# Print the Score
print("Test score: {0:.2f} %".format(100 * score))  


Test score: 96.95 %


In [12]:
# Predict the Labels using the reloaded Model
y_pred_tuned = joblib_SVM_model.predict(X_test_partial_scaled)

# Evaluate the tuned model with various metrics
mae_tuned = mean_absolute_error(y_test, y_pred_tuned)
mse_tuned = mean_squared_error(y_test, y_pred_tuned)
rmse_tuned = mean_squared_error(y_test, y_pred_tuned, squared=False)
r2_tuned = r2_score(y_test, y_pred_tuned)
mape_tuned = (abs((y_test - y_pred_tuned) / y_test).mean()) * 100
evs_tuned = explained_variance_score(y_test, y_pred_tuned)
max_err_tuned = max_error(y_test, y_pred_tuned)

# Print the evaluation metrics for the tuned model
print(f"SVM model hyperparameter tuning:")
print(f"Tuned SVM Mean Absolute Error (MAE): {mae_tuned:.4f}")
print(f"Tuned SVM Mean Squared Error (MSE): {mse_tuned:.4f}")
print(f"Tuned SVM Root Mean Squared Error (RMSE): {rmse_tuned:.4f}")
print(f"Tuned SVM R-squared (R2): {r2_tuned:.4f}")
print(f"Tuned SVM Mean Absolute Percentage Error (MAPE): {mape_tuned:.4f}%")
print(f"Tuned SVM Explained Variance Score: {evs_tuned:.4f}")
print(f"Tuned SVM Max Error: {max_err_tuned:.4f}")


SVM model hyperparameter tuning:
Tuned SVM Mean Absolute Error (MAE): 21.3325
Tuned SVM Mean Squared Error (MSE): 882.3109
Tuned SVM Root Mean Squared Error (RMSE): 29.7037
Tuned SVM R-squared (R2): 0.9695
Tuned SVM Mean Absolute Percentage Error (MAPE): 5.6417%
Tuned SVM Explained Variance Score: 0.9696
Tuned SVM Max Error: 297.4948


#### KNN

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error, explained_variance_score, max_error

def custom_weight(distances):
    return 1 / (distances + 1e-5) 

# Define the parameter grid for tuning
param_grid = {
    'n_neighbors': [3, 5, 7, 10, 15], 
    'weights': ['uniform', custom_weight], 
    'metric': ['euclidean', 'manhattan', 'minkowski']  
}

# Set up GridSearchCV
grid_search_knn = GridSearchCV(
    KNeighborsRegressor(),
    param_grid,
    scoring='neg_mean_squared_error',  
    cv=3,  
    verbose=1
)

# Fit GridSearchCV to the training data
grid_search_knn.fit(X_train_partial_scaled, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search_knn.best_params_)
print("Best cross-validation score (negative MSE): ", grid_search_knn.best_score_)

# Retrieve the best model from GridSearchCV
best_knn_model = grid_search_knn.best_estimator_

best_knn_model.fit(X_train_partial_scaled, y_train) 


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters found:  {'metric': 'manhattan', 'n_neighbors': 15, 'weights': <function custom_weight at 0x0000013B6774A980>}
Best cross-validation score (negative MSE):  -1587.5531463635853


KNeighborsRegressor(metric='manhattan', n_neighbors=15,
                    weights=<function custom_weight at 0x0000013B6774A980>)

In [15]:
# Print the best parameters and the best score
print("Best parameters found: ", grid_search_knn.best_params_)
print("Best cross-validation score (negative MSE): ", grid_search_knn.best_score_)

Best parameters found:  {'metric': 'manhattan', 'n_neighbors': 15, 'weights': <function custom_weight at 0x0000013B6774A980>}
Best cross-validation score (negative MSE):  -1587.5531463635853


In [16]:
import joblib

class ModelWrapper:
    def __init__(self, model, custom_function):
        self.model = model
        self.custom_function = custom_function

wrapper = ModelWrapper(best_knn_model, custom_weight)
joblib.dump(wrapper, "KNN_model.pkl")


['KNN_model.pkl']

In [17]:
import joblib

class ModelWrapper:
    def __init__(self, model, weight_function):
        self.model = model
        self.custom_function = custom_function

def custom_weight(distances):
    return 1 / (distances + 1e-5)

# Load the model
wrapper = joblib.load("KNN_model.pkl")
knn_model = wrapper.model
custom_function = wrapper.custom_function

print(knn_model)
print(custom_function)


KNeighborsRegressor(metric='manhattan', n_neighbors=15,
                    weights=<function custom_weight at 0x0000013B62ABEFC0>)
<function custom_weight at 0x0000013B62ABEFC0>


In [18]:
# Use the Reloaded Joblib Model to 
# Calculate the accuracy score and predict target values

# Calculate the Score 
score_KNN = knn_model.score(X_test_partial_scaled, y_test)  
# Print the Score
print("Test score: {0:.2f} %".format(100 * score_KNN))  

# Predict the Labels using the reloaded Model
Ypredict_KNN = knn_model.predict(X_test_partial_scaled)  

Ypredict_KNN

Test score: 95.24 %


array([408.53502862, 406.48797356, 406.62202912, ..., 191.04572436,
       190.31431416, 190.10264566])

In [19]:
# Make predictions on the test set
y_pred_knn_tuned = knn_model.predict(X_test_partial_scaled)

# Calculate the evaluation metrics
mae_knn = mean_absolute_error(y_test, y_pred_knn_tuned)
mse_knn = mean_squared_error(y_test, y_pred_knn_tuned)
rmse_knn = mean_squared_error(y_test, y_pred_knn_tuned, squared=False)
r2_knn = r2_score(y_test, y_pred_knn_tuned)
mape_knn = (abs((y_test - y_pred_tuned) / y_test).mean()) * 100  # Convert to percentage
evs_knn = explained_variance_score(y_test, y_pred_knn_tuned)
max_err_knn = max_error(y_test, y_pred_knn_tuned)

# Print the evaluation metrics
print(f"KNN Mean Absolute Error (MAE): {mae_knn:.4f}")
print(f"KNN Mean Squared Error (MSE): {mse_knn:.4f}")
print(f"KNN Root Mean Squared Error (RMSE): {rmse_knn:.4f}")
print(f"KNN R-squared (R2): {r2_knn:.4f}")
print(f"KNN Mean Absolute Percentage Error (MAPE): {mape_knn:.4f}%")
print(f"KNN Explained Variance Score: {evs_knn:.4f}")
print(f"KNN Max Error: {max_err_knn:.4f}")

KNN Mean Absolute Error (MAE): 26.8363
KNN Mean Squared Error (MSE): 1377.9526
KNN Root Mean Squared Error (RMSE): 37.1208
KNN R-squared (R2): 0.9524
KNN Mean Absolute Percentage Error (MAPE): 5.6417%
KNN Explained Variance Score: 0.9555
KNN Max Error: 339.2431


### Result comparision and disscussion

In [20]:
import pandas as pd

# Define the evaluation metrics for SVM with RBF kernel
metrics_svm = {
    'Metric': ['Mean Absolute Error (MAE)', 'Mean Squared Error (MSE)', 'Root Mean Squared Error (RMSE)', 
               'R^2 Score (R2)', 'Mean Absolute Percentage Error (MAPE)', 'Explained Variance Score (EVS)'],
    'SVM (RBF Kernel)': [mae_tuned, mse_tuned, rmse_tuned, r2_tuned, mape_tuned, evs_tuned]
}

# Define the evaluation metrics for KNN
metrics_knn = {
    'Metric': ['Mean Absolute Error (MAE)', 'Mean Squared Error (MSE)', 'Root Mean Squared Error (RMSE)', 
               'R^2 Score (R2)', 'Mean Absolute Percentage Error (MAPE)', 'Explained Variance Score (EVS)'],
    'KNN': [mae_knn, mse_knn, rmse_knn, r2_knn, mape_knn, evs_knn]
}

# Create DataFrames
df_svm = pd.DataFrame(metrics_svm)
df_knn = pd.DataFrame(metrics_knn)

# Merge DataFrames for a complete table
df_results = pd.merge(df_svm, df_knn, on='Metric')

# Display the table
print(df_results)


                                  Metric  SVM (RBF Kernel)          KNN
0              Mean Absolute Error (MAE)         21.332516    26.836255
1               Mean Squared Error (MSE)        882.310904  1377.952618
2         Root Mean Squared Error (RMSE)         29.703719    37.120784
3                         R^2 Score (R2)          0.969495     0.952359
4  Mean Absolute Percentage Error (MAPE)          5.641681     5.641681
5         Explained Variance Score (EVS)          0.969602     0.955501
